In [1]:
import os
import matplotlib as mpl
import matplotlib.pylab as plt
import itertools
import csv , codecs
import glob
from PIL import Image
import numpy as np
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn
from sklearn import decomposition
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [2]:
data_dir = "./EEG_data/"

## CSV 파일 생성 함수

In [3]:
def EEG_data_csv(data_dir):
    
    # Kinds of Power 입력
    file_list = os.listdir(data_dir)
    file_list.sort()
       
    global kinds_of_power
    kinds_of_power = "Relative"
    
    excel_dir = "./EEG_data" + '/' + kinds_of_power
    excel_list = sorted(glob.glob(excel_dir+'/[!~]*.xlsx'))
    
    # EEG_csv_data 디렉토리 생성
    csv_dir = "./EEG_csv_data"
    if not os.path.isdir(csv_dir):
        os.mkdir(csv_dir)
    
    Abs_dir ="./EEG_csv_data/Abs"
    Relative_dir = "./EEG_csv_data/Relative"
    
    if not os.path.isdir(Abs_dir):
        os.mkdir(Abs_dir)
    if not os.path.isdir(Relative_dir):
        os.mkdir(Relative_dir)
    
    book = Workbook()
    sheet = book.active
    
    # x축,y축 생성
    for i in range (1,21) :
        if i < 10 :
            sheet['A'+str(i+1)] = 'S0'+str(i)
        if i == 10 :
            sheet['A'+str(i+1)] = 'S'+str(10)
        if i > 10 :
            sheet['A'+str(i+1)] = 'S'+str(i)
            
    for j in range (0,12):
        for i in range(1,33):
            sheet.cell(row=1, column=160*j+i+1).value = 'ACh' + str(i)
            sheet.cell(row=1, column=160*j+i+33).value = 'BCh' + str(i)
            sheet.cell(row=1, column=160*j+i+65).value = 'DCh' + str(i)
            sheet.cell(row=1, column=160*j+i+97).value = 'GCh' + str(i)
            sheet.cell(row=1, column=160*j+i+129).value = 'TCh' + str(i)

    # 파일을 순차적으로 열어서 셀 영역을 복사
    # j: 엑셀 파일의 수, k: 각 엑셀 파일 당 시트의 수, n: task의 수, l: subject의 수, m: ch1-ch32

    for j in range(0,5):
        wb = openpyxl.load_workbook(excel_list[j], data_only=True)
        wb_sheet = wb.sheetnames
        for k in range(0,12):
            source = wb[wb_sheet[k]]
            for l in range(2, 22):
                for m in range(2, 34):
                    sheet.cell(row=l,column=160*k+m+32*j).value = source.cell(row=l,column=m).value

    # EEG_csv_data 디렉토리에 csv 파일로 저장
    if kinds_of_power == "Abs":
        with open(Abs_dir + str('/') +'Abs_Merged.csv', 'w', newline="") as f:
            c = csv.writer(f)
            for r in sheet.rows:
                c.writerow([cell.value for cell in r])
    elif kinds_of_power == "Relative":
        with open(Relative_dir + str('/') +'Relative_Merged.csv', 'w', newline="") as f:
            c = csv.writer(f)
            for r in sheet.rows:
                c.writerow([cell.value for cell in r])

## CSV 파일에서 Pandas dataframe으로 가져오는 함수

In [4]:
EEG_data_csv(data_dir)


### task 0 1 2 : 1-back,  2-back,  rest

### eye 0  1    : close eye,     open eye

### respiratory 0 1 2 : mouse, nose, O2 



In [5]:
def dataset_csv_3():
    normalize_power = str(input("Relative, Abs :"))
    excel_dir = "./EEG_data" + '/' + normalize_power
    excel_list = sorted(glob.glob(excel_dir+'/[!~]*.xlsx'))
    
    subject = int(input("subject :"))
    book = Workbook()
    sheet = book.active
    sheet.cell(row=1, column=161).value = 'target'
 
    # x축 생성   
    for i in range(1,33):
        sheet.cell(row=1, column=i).value = 'ACh' + str(i)
        sheet.cell(row=1, column=i+32).value = 'BCh' + str(i)
        sheet.cell(row=1, column=i+64).value = 'DCh' + str(i)
        sheet.cell(row=1, column=i+96).value = 'GCh' + str(i)
        sheet.cell(row=1, column=i+128).value = 'TCh' + str(i)
        
    # 파일을 순차적으로 열어서 셀 영역을 복사
    for j in range(0,5):
        wb = openpyxl.load_workbook(excel_list[j], data_only=True)
        wb_sheet = wb.sheetnames
        for k in range(0,12):
            source = wb[wb_sheet[k]]
            for l in range(2, 22):
                for m in range(2, 34):
                    sheet.cell(row=20*k+l,column=32*j+m-1).value = source.cell(row=l,column=m).value
        

    with open('./dataset_eeg_3.csv', 'w', newline="") as f:
        c = csv.writer(f)
        for r in sheet.rows:
            c.writerow([cell.value for cell in r])
            
    df = pd.read_csv('./dataset_eeg_3.csv')
    df[df =='                      NaN'] =np.nan
    df = df.fillna(0.0).astype('float64')
    pd.options.mode.chained_assignment = None

    df["target"][0*subject:1*subject] = "0"
    df["target"][1*subject:2*subject] = "1"
    df["target"][2*subject:3*subject] = "2"

    df["target"][3*subject:4*subject] = "0"
    df["target"][4*subject:5*subject] = "1"
    df["target"][5*subject:6*subject] = "2"

    df["target"][6*subject:7*subject] = "0"
    df["target"][7*subject:8*subject] = "1"
    df["target"][8*subject:9*subject] = "2"

    df["target"][9*subject:10*subject] = "0"
    df["target"][10*subject:11*subject] = "1"
    df["target"][11*subject:12*subject] = "2"
    df.astype('float64')
    df.to_csv('./dataset_eeg_3.csv')
    
    return df
            


In [6]:
eeg_data_3 = dataset_csv_3()

Relative, Abs : Relative
subject : 20


C:\Users\hyg44\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [7]:


ds_train = {}
ds_test = {}


# train : open_eye + 1-back
ds_train['open_eye_1back']= eeg_data_3.iloc[0:40]
ds_test['open_eye_1back'] = eeg_data_3.iloc[40:60]

# train : open_eye + 2-back
ds_train['open_eye_2back']= eeg_data_3.iloc[60:100]
ds_test['open_eye_2back']= eeg_data_3.iloc[100:120]

# train : open_eye + rest 
ds_train['open_eye_rest']= eeg_data_3.iloc[180:220]
ds_test['open_eye_rest']= eeg_data_3.iloc[220:240]

# train : close_eye + rest
ds_train['close_eye_rest']=eeg_data_3.iloc[120:160]
ds_test['close_eye_rest']=eeg_data_3.iloc[160:180]

# train : total
ds_train['total']=eeg_data_3.iloc[0:40].append( eeg_data_3.iloc[60:100]).append(eeg_data_3.iloc[120:160]).append(eeg_data_3.iloc[180:220])
ds_test['total']= eeg_data_3.iloc[40:60].append( eeg_data_3.iloc[100:120]).append(eeg_data_3.iloc[160:180]).append(eeg_data_3.iloc[220:240])



In [8]:
def M_N_O2 (tasks, k_parameter, num_iter):

    for mode in ['no_lda','lda']:
        for task_train,task_test in tasks:

            print('-------------------')
            print('training:{}/test:{}/mode:{}'.format(task_train,task_test,mode))
            train_3 = ds_train[task_train]
            test_3  = ds_test[task_test]

            X = np.array(train_3.iloc[:,0:-1])
            Y = np.array(train_3['target'])

            rnd_clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)

            all_scores = []
            Mouse = 0
            Nose = 0
            for idx in range(num_iter):
                skf = StratifiedKFold(n_splits=k_parameter, shuffle=True)
                skf.get_n_splits(X,Y)

                for train_index, test_index in skf.split(X,Y):

                    X_final = X
                    lda = LinearDiscriminantAnalysis(n_components=1)
                    if(mode == 'lda') : 
                        lda.fit(X,Y)
                        X_final = lda.transform(X)

                    rnd_clf.fit(X_final[train_index], Y[train_index])
                    val_mae = rnd_clf.score(X_final[test_index],Y[test_index])
                    all_scores.append(val_mae)

                    test = np.array(test_3.iloc[:,0:-1])
                    if mode == 'lda':
                        test = lda.transform(test)

                    aaa = rnd_clf.predict(test)

                    Mouse = Mouse+np.count_nonzero(aaa=='0')
                    Nose = Nose+ np.count_nonzero(aaa=='1')

            print('model_score:{}'.format(np.mean(all_scores)))
            print('mouse:{}({:.2}), nose:{}({:.2}) / total:{}'.format(Mouse,Mouse/(Mouse+Nose),Nose,Nose/(Mouse+Nose),(Mouse+Nose)))


In [9]:

tasks5 = [
     ('close_eye_rest','close_eye_rest'),
     ('close_eye_rest','open_eye_rest'),
     ('close_eye_rest','open_eye_1back'),
     ('close_eye_rest','open_eye_2back'),
     ('close_eye_rest','total')
 ]


#tasks6 = [
#     ('close_eye_rest','open_eye_rest'),
#     ('close_eye_rest','open_eye_1back'),
#     ('close_eye_rest','open_eye_2back'),
# ]

In [ ]:
M_N_O2(tasks5,k_parameter = 10,num_iter = 1)

-------------------
training:close_eye_rest/test:close_eye_rest/mode:no_lda
model_score:0.15
mouse:85(0.42), nose:115(0.57) / total:200
-------------------
training:close_eye_rest/test:open_eye_rest/mode:no_lda
model_score:0.15
mouse:63(0.32), nose:137(0.69) / total:200
-------------------
training:close_eye_rest/test:open_eye_1back/mode:no_lda
model_score:0.15
mouse:37(0.18), nose:163(0.81) / total:200
-------------------
training:close_eye_rest/test:open_eye_2back/mode:no_lda
model_score:0.25
mouse:53(0.27), nose:147(0.73) / total:200
-------------------
training:close_eye_rest/test:total/mode:no_lda
model_score:0.175
mouse:248(0.31), nose:552(0.69) / total:800
-------------------
training:close_eye_rest/test:close_eye_rest/mode:lda
